In [1]:
import os 
import numpy as np
import re

In [2]:
usc_dir = "/home/grilio/Documents/SLP/Lab_2/kaldi/egs/usc"

In [5]:
os.getcwd()

'/home/grilio/Documents/SLP/Lab_2/kaldi/egs/usc'

In [46]:
os.chdir(usc_dic)
os.getcwd()

'/home/grilio/Documents/SLP/Lab_2/kaldi/egs/usc'

# 3 Preparatory Steps

In [12]:
os.mkdir('data')

folders = ['./data/dev','./data/train','./data/test']
for folder in folders:
    try:
        os.mkdir(folder)
    except OSError as error: 
        print(error)

In [13]:
with open('./lexicon.txt','r') as f:
    lexicon = {}
    lines = f.readlines()
    print(lines)
    for line in lines:
        key = line.split('\t')[0]
        value = line.split('\t')[1].replace('\n', '').strip()
        lexicon[key] = value

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
dirs = ['train','dev','test'] 
for direc in dirs:
    
    if(direc == 'train'):
        path = 'training'
    elif(direc == 'dev'):
        path = 'validation'
    else:
        path = 'testing'
        
    uttids = []
    with open('./filesets/{}.txt'.format(path),'r') as f:
        speakers = f.readlines()
        speakers = [speaker.strip('\n') for speaker in speakers]
        for speaker in speakers:
            uttids.append(speaker)

    with open('./transcriptions.txt','r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            line = re.sub(r"[^a-zA-Z ']+", '', lines[i]).lower()
            lines[i] = line

    with open('./data/{}/utt2spk'.format(direc),'w') as f:
        for uttid in uttids:
            f.write("{} {}\n".format(uttid,uttid[0:2]))

    with open('./data/{}/wav.scp'.format(direc),'w') as f:
        for uttid in uttids:
            f.write("{} {}\n".format(uttid,'./wav/{}.wav'.format(uttid)))

    with open('./data/{}/text'.format(direc),'w') as f:
        for uttid in uttids:
            line = lines[int(uttid[-3:])-1].split(' ')
            start = 'sil'
            for word in line:
                start = start + ' ' + lexicon[word.upper()]
            start = start + ' sil'
            f.write("{} {}\n".format(uttid,start))

# 4 Main lab steps

## 4.1 Preparing the USC-TIMIT speech recognition system

5. Create the folder local and create a soft link to the file `steps/score_kaldi.sh` inside `local`.

In [31]:
os.chdir(usc_dir + "/data")

In [15]:
folder = './local'
os.mkdir(folder)

In [31]:
os.chdir(usc_dir + "/data/local")

In [33]:
# change the user
! ln -s /home/grilio/Documents/SLP/Lab_2/kaldi/egs/usc/steps/score_kaldi.sh score_kaldi_link

6. Create the folder `conf` and copy inside it the file `mfcc.conf` provided to you in the helper code.

In [37]:
folder = './conf'
os.mkdir(folder)

In [47]:
! wget https://github.com/slp-ntua/slp-labs/blob/master/lab2/mfcc.conf

--2023-05-06 20:58:51--  https://github.com/slp-ntua/slp-labs/blob/master/lab2/mfcc.conf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘mfcc.conf’

mfcc.conf               [ <=>                ] 143,79K  --.-KB/s    in 0,08s   

2023-05-06 20:58:52 (1,71 MB/s) - ‘mfcc.conf’ saved [147244]



7. Finally, create the following folders: `data/lang`, `data/local/dict`, `data/local/lm_tmp`, `data/local/nist_lm`.

In [39]:
os.chdir(usc_dir)

In [40]:
folders = ['./data/lang','./data/local/dict', './data/local/lm_tmp', './data/local/nist_lm']
for folder in folders:
    try:
        os.mkdir(folder)
    except OSError as error: 
        print(error)

## 4.2 Building the language model

1. Inside the folder `data/local/dict` you will put the files needed for constructing the language model.

- Create `silence_phones.txt` and `optional_silence.txt` containing only the silence phoneme (`sil`).

In [41]:
with open('./data/local/dict/silence_phones.txt','w') as f:
    f.write("sil\n")
with open('./data/local/dict/optional_silence.txt','w') as f:
    f.write("sil\n")    

- Create `nonsilence_phones.txt`, which contains all other phonemes (one per line, sorted).

- Create `lexicon.txt` containing the vocabulary. Since we perform phoneme recognition (and not word recognition), `lexicon.txt` contains a 1-1 mapping of each phoneme to itself. In each line of `lexicon.txt` you must put a phoneme then a `<space>` and then the same phoneme again. Do not forget to include the silence phoneme (`sil`).

In [42]:
phonemes = []
with open('./lexicon.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        key = line.split('\t')[0]
        value = line.split('\t')[1].replace('\n', '').strip().split(' ')
        for phoneme in value:
            if(phoneme not in phonemes):
                phonemes.append(phoneme)
                
phonemes.sort()       
with open('./data/local/dict/nonsilence_phones.txt','w') as f:
    for phoneme in phonemes:
        if(phoneme != "sil"):
            f.write("{}\n".format(phoneme))

with open('./data/local/dict/lexicon.txt','w') as f:
    for phoneme in phonemes:
        if(phoneme != '<oov>'):
            f.write("{} {}\n".format(phoneme,phoneme))
        else:
            f.write("{} {}\n".format(phoneme,'sil'))

- To create the `lm_train.text`, you need to add special tokens `<s>` and `</s>` in the beginning and the end of each line respectively, based on the file text that you created during the lab preparation. Similarly, you also need to create `lm_dev.text` and `lm_test.text` files in the same way.

In [43]:
dirs = ['train','dev','test'] 
for direc in dirs:
    
    if(direc == 'train'):
        path = 'training'
    elif(direc == 'dev'):
        path = 'validation'
    else:
        path = 'testing'
        
    uttids = []
    with open('./filesets/{}.txt'.format(path),'r') as f:
        speakers = f.readlines()
        speakers = [speaker.strip('\n') for speaker in speakers]
        for speaker in speakers:
            uttids.append(speaker)

    with open('./transcriptions.txt','r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            line = re.sub(r"[^a-zA-Z ']+", '', lines[i]).lower()
            lines[i] = line
            
    with open('./data/local/lm_tmp/lm_{}.txt'.format(direc),'w') as f:
        for uttid in uttids:
            line = lines[int(uttid[-3:])-1].split(' ')
            start = 'sil'
            for word in line:
                start = start + ' ' + lexicon[word.upper()]
            start = start + ' sil'
            f.write("{} <s> {} </s>\n".format(uttid,start))   

- Finally, create an empty file `extra_questions.txt`.


In [44]:
f = open('./data/extra_questions.txt','x')